<a href="https://colab.research.google.com/github/xr77/desktop-tutorial/blob/master/Vinci_Booher_dense_longitudinal_flux2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Dense Longitudinal Neuroimaging: Temporally Precise Modeling of Brain Change Trajectories**

---
https://bit.ly/dense-longitudinal




# **Step 0**: Before we start, let's prepare the data and compute environment.

In [ ]:
# Download tutorial data from OSF, including functional and motion data for 12 visits from 1 participant.
# This will take 5-7 minutes, depending on download speed.
!wget https://files.osf.io/v1/resources/qz6dp/providers/osfstorage/?zip= -O /content/osf_qz6dp.zip
!unzip -o "/content/osf_qz6dp.zip" -d "/content/dense_longitudinal_data"
print(f"✅ Downloaded to /content/dense_longitudinal_data")

In [ ]:
# Install GLMsingle and nilearn.
!pip install git+https://github.com/cvnlab/GLMsingle.git
!pip install nilearn

# Import necessary packages.
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import nibabel as nib
import requests
import glob
import os
from os.path import join, exists
from IPython.display import YouTubeVideo
from IPython.display import Image
from scipy import interpolate

import time
import warnings
from pprint import pprint
from pathlib import Path
import h5py
import random

from glmsingle.glmsingle import GLM_single

In [ ]:
# Define internal functions to be used later.

# Define an internal function that will convert our _events.tsv file to a sparse design matrix.
def tsv_to_design(events_path, T, tr, use_duration=False, rounding='rint'):
    df = pd.read_csv(events_path, sep='\t')
    df.columns = [c.strip() for c in df.columns]
    if 'onset' not in df.columns:
        raise ValueError(f"{events_path} missing 'onset'")
    df = df.dropna(subset=['onset']).sort_values('onset').copy()

    on = df['onset'].astype(float).to_numpy()

    if use_duration and 'duration' in df.columns:
        dur = df['duration'].astype(float).fillna(0.0).to_numpy()
        dur = np.clip(dur, 0, None)
        widths = np.maximum(1, np.ceil(dur / tr).astype(int))
        idx = np.floor(on / tr).astype(int)   # align to start TR for multi-TR blocks
    else:
        widths = np.ones_like(on, dtype=int)  # single-TR impulse
        mapper = {'rint': np.rint, 'floor': np.floor, 'ceil': np.ceil}
        idx = mapper.get(rounding, np.rint)(on / tr).astype(int)

    idx = np.clip(idx, 0, T - 1)

    M = np.zeros((T, len(on)), dtype=np.float32)
    for j, r0 in enumerate(idx):
        a, b = max(0, r0), min(T, r0 + widths[j])
        if a < b:
            M[a:b, j] = 1.0
    return M

# Define time series interpolation function.
def t_series_interpolation(data, old_tr, new_tr, *, fill_value="extrapolate", dtype=np.float32):
    """
    Interpolate data from old_tr to new_tr.
    data: 4D (X, Y, Z, T)
    old_tr, new_tr: second
    fill_value: 'extrapolate'
    dtype: data type
    """
    n_vol = data.shape[-1]
    if n_vol < 2 or old_tr == new_tr: # cannot perform interpolation if T < 2
        return data.astype(dtype, copy=True)

    # create the old time series
    old_t = np.arange(n_vol) * old_tr
    t_end = (n_vol - 1) * old_tr

    # create new time series
    n_new = int(np.ceil(n_vol * old_tr / new_tr))
    new_t = np.linspace(0.0, (n_new - 1) * new_tr, n_new)

    # set the interpolation
    f = interpolate.interp1d(
        old_t, data, kind="linear", axis=-1,
        bounds_error=False, fill_value=fill_value, assume_sorted=True
    )
    out = f(new_t).astype(dtype, copy=False)
    return out

# Define function to plot ROI results across visits.
def plot_roi_results(m, md, sd, se, roi_name):

    """Plot ROI results across visits - displays plot and saves to file."""

    fig, ax = plt.subplots(figsize=(6, 4))

    xval = m.index+1  # Visit numbers

    # Zero line
    gray = [0.5, 0.5, 0.5]
    ax.axhline(0, linestyle=':', color=gray)

    # Plot faces data
    # color definitions
    fcolor = [0.94, 0.5, 0.5]    # Faces
    ax.errorbar(xval, m['Faces'], se['Faces'], color=fcolor, linewidth=5, capsize=0)
    ax.scatter(xval, m['Faces'], color=fcolor, s=300, edgecolors='white')
    ax.axhline(y=m['Faces'].mean(), color=fcolor + [0.6], linewidth=1)

    # Plot letters data
    lcolor = [0, 0.5, 1]         # Letters
    ax.errorbar(xval, m['Letters'], se['Letters'], color=lcolor, linewidth=5, capsize=0)
    ax.scatter(xval, m['Letters'], color=lcolor, s=300, edgecolors='white')
    ax.axhline(y=m['Letters'].mean(), color=lcolor + [0.6], linewidth=1)

    # Axis settings
    ax.set_xlim(0.5, 12.5)
    ax.set_xticks(np.arange(1,13))
    ax.set_xlabel('Month', fontsize=16)
    ax.set_ylabel('Beta Value', fontsize=16)
    ax.set_title(roi_name, fontsize=18)

    # Legend
    ax.legend(['', 'Faces', '', 'Letters', ''], loc='upper right', frameon=False)

    plt.tight_layout()
    plt.show()

    # Save
    save_path = os.path.join(outdir, f'plot_{roi_name}.png')
    fig.savefig(save_path, bbox_inches='tight', dpi=300)
    print(f"Plot saved: {save_path}")

# Define function to align beta and ROI dimensions (e.g., cropping in Z).
def align_xyz(beta3d, mask3d, tag=""):
    if beta3d.shape[:2] != mask3d.shape[:2]:
        raise ValueError(f"[{tag}] X/Y mismatch: beta {beta3d.shape} vs ROI {mask3d.shape}")
    if beta3d.shape[2] != mask3d.shape[2]:
        z_new = min(beta3d.shape[2], mask3d.shape[2])
        print(f"⚠️ [{tag}] Cropping Z to {z_new} (beta Z={beta3d.shape[2]}, ROI Z={mask3d.shape[2]})")
        beta3d = beta3d[:, :, :z_new]
        mask3d = mask3d[:, :, :z_new]
    return beta3d, mask3d

# Define simple helper function help with reading rootdir.
def visit_dir(v):
    """Return full path to a visit directory"""
    return Path(rootdir) / "function" / f"sub-03{v}"

# Define configuration function to handle different indexing between Matlab
# (i.e., 1 indexing) and Python (i.e., 0 indexing).
def load_modelmd(filepath):
    """Load and reshape beta map from .mat file"""
    with h5py.File(filepath, 'r') as f:
        data = np.array(f['modelmd'][()])

        # Fix dimension order from MATLAB to (X,Y,Z,T)
        expected_shape = (58, 73, 57, 208) # this is specific to the sample dataset
        if data.ndim == 3:
            data = np.transpose(data, (2, 1, 0))  # (Y,Z,T) → (Z,Y,T) → needs pad
            data = np.pad(data, [
                (0, expected_shape[1] - data.shape[0]),
                (0, expected_shape[2] - data.shape[1]),
                (0, 0)
            ])
            data = np.expand_dims(data, axis=0)  # add X dimension
        elif data.ndim == 4:
            data = np.transpose(data, (3, 2, 1, 0))  # (T,Z,Y,X) → (X,Y,Z,T)

    return data

# Define a function to load in design information outpus from GLMsingle and
# configure for Python.
def load_designinfo(filepath):
    """Load design info: stimorder, tr, stimdur"""
    with h5py.File(filepath, 'r') as f:
        return {
            'stimorder': np.array(f['stimorder'][()]).flatten(),
            'tr': float(f['tr'][()][0, 0]),
            'stimdur': float(f['stimdur'][()][0, 0])
        }

# Some background while we wait...

**Dense longitudinal neuroimaging** is a type of precision neuroimaging where the precision is applied to the sampling of the change trajectory -- an approach that is particularly relevant for studies of development and learning.

In [ ]:
Image(url="https://raw.githubusercontent.com/Learning-and-Neurodevelopment-Lab/dense-longitudinal-tutorial/refs/heads/main/fig-1.png", width=800)

In [ ]:
Image(url="https://raw.githubusercontent.com/Learning-and-Neurodevelopment-Lab/dense-longitudinal-tutorial/refs/heads/main/fig-2.png", width=800)

This tutorial analyzes dense longitudinal neuroimaging data to **quantify changes in brain activation within the visual word form area (VWFA) and the fusiform face area (FFA) throughout first grade**. More specifically, we will analyze task-based fMRI data from one child's brain sampled every month throughout their first grade year.

The analysis approach is **an individualized ROI analysis**: Identify FFA AND VWFA using a standard localizer. Then, use a different localizer to extract estimates of brain responses from those ROIs every month from the beginning to the end of first grade.

# **Step 1**: Collect dense longitudinal data.

**Goal**: Acquire dense longitudinal neuroimaging data that we can use to estimate changes in brain responses within the VWFA and FFA throughout the first grade.  

**We will**: Discuss (1) the experimental paradigm, (2) practical considerations related to retention, and (3) an evaluation of motion for dense longitudinal neuroimaging data.

In [ ]:
# (1) THE EXPERIMENTAL PARADIGM.

We will use our ([SS](https://github.com/Learning-and-Neurodevelopment-Lab/Sesame_Street_Cohort1)) paradigm that **presents participants with images from Sesame Street**. Only one image is presented per trial so that each trial can be labeled for different object categories, depending on the experimenter's needs. There are approximately 208 unique images presented at each visit.

In [ ]:
YouTubeVideo("Tb37rSZghNQ", embed=True, width=640, height=320)

For this tutorial, **we're labeling each image (i.e., trial) for the presence or absence of faces and letters** so that we can use the data to measure face- and letter-selectivity within the VWFA and FFA.

In [ ]:
# (2) PRACTICAL CONSIDERATIONS RELATED TO RETENTION.

In [ ]:
Image(url="https://raw.githubusercontent.com/Learning-and-Neurodevelopment-Lab/dense-longitudinal-tutorial/refs/heads/main/fig-3.png", width=400)

There were 12 visits that occurred monthly, starting the month before first grade and ending the month after first grade.

In [ ]:
# (3) AN EVALUATION OF MOTION FOR DENSE LONGITUDINAL NEUROIMAGING DATA.

# Set basic variables.
subid = 'sub-03'
visits = ['01','02','03','04','05','06','07','08','09','10','11','12']
rootdir = '/content/dense_longitudinal_data/'

# Define the folder where motion data is located
data_folder = os.path.join(rootdir, "motion")

# Initialize lists to store the mean and standard deviation for each month
means = []
stds = []

# Loop through the 12 months (visits)
for month in range(1, 13):
    subject_month_id = f"03{month:02d}"

    # This list will hold the mean value from each of the 4 runs
    run_means = []

    # Loop through the 4 runs for the current month
    for run in range(1, 5):

        file_name = f"sub-{subject_month_id}_task-ss_run-0{run}_desc-confounds_timeseries.tsv"
        file_path = os.path.join(data_folder, file_name)

        if os.path.exists(file_path):
            df = pd.read_csv(file_path, sep='\t')
            if 'framewise_displacement' in df.columns and not df['framewise_displacement'].dropna().empty:
                run_means.append(df['framewise_displacement'].mean())
        else:
            print(f"File not found: {file_path}")

    if run_means:
        run_means_series = pd.Series(run_means)
        means.append(run_means_series.mean())
        stds.append(run_means_series.std())
    else:
        means.append(None)
        stds.append(None)

# Create the plot
months = range(1, 13)
plt.errorbar(months, means, yerr=stds, fmt='-o', capsize=5, label='Monthly Mean with Std Dev')

# Calculate and plot the overall mean/std
if pd.Series(means).notna().any():
    grand_mean = pd.Series(means).mean()
    grand_std = pd.Series(means).std()

    plt.axhline(y=grand_mean, color='r', linestyle='--', label='Mean across 12 Months')
    plt.axhspan(grand_mean - grand_std, grand_mean + grand_std,
                color='r', alpha=0.2, label='SD across 12 Months')

# Add labels and title
plt.xlabel("Month")
plt.ylabel("Motion (framewise displacement in mm)")
plt.title("Subject 03")
plt.xticks(months)
plt.legend()
plt.grid(True)
plt.ylim(0, 3)

# Display the plot
plt.show()

# **Step 2**: Estimate responses for face and letter images at all visits.

**Goal**: Calculate beta-weights to estimate brain responses to face and letter images at all visits.   

**We will**: (1) Prepare SS design matrix → (2) Prepare the SS functional data → (3) Run GLMsingle → (4) Visualize data quality outputs.

We use **GLMsingle to get trial-specific beta-weights**, meaning that we get a measure of activation to each image (i.e., trial). Using GLMsingle with the SS design allows analyses beyond face- and letter responses. We can select any category of interest, for example, and then relabel the trials for our new category of interest.

### (1) Prepare SS design matrix.

In [ ]:
# Define variables for GLMsingle.

tr = 1.0        # expected length of tr, required input to GLMsingle
nvol = 294      # expected number of volumes when length of TR is set to 1.0
stimdur  = 3    # stimulus duration, required input to GLMsingle

In [ ]:
## CONVERT BIDS-READY _EVENTS.TSV TO A GLMSINGLE-READY DESIGN MATRIX.

# To estimate brain responses to single images, the design matrix should contain
# one predicter for each unique image. Design Matrix Shape: (T, C).
# T: Number of TRs. C: Number of unique images shown.

# Define _event.tsv file locations.
derivatives_path = f'{rootdir}/function/sub-0312/func'

# find events files (one per run)
events_files = sorted(glob.glob(os.path.join(derivatives_path, f'*task-ss*_events.tsv')))

design = []
design = [tsv_to_design(evt, nvol, tr, use_duration=False, rounding='rint')
          for evt in events_files]

print(f"Found {len(events_files)} runs.")
print("run 1 design shape:", design[0].shape)
print("run 2 design shape:", design[1].shape)

In [ ]:
# plot first run (imshow)
plt.figure(figsize=(5,5))
plt.imshow(design[0], aspect='auto', origin='lower', interpolation='nearest')
plt.title(f"design (run 1)  T={design[0].shape[0]}, trials={design[0].shape[1]}")
plt.xlabel('Condition (i.e., image)'); plt.ylabel('TR (after interpolation)')
plt.colorbar(label='0/1')
plt.show()

 ### (2) Prepare the SS functional data.

Note that all of the data we are working with have already been minimally preprocessed using [fMRIprep](https://fmriprep.org/en/stable/), and this tutorial won't cover the preprocessing steps.

In [ ]:
## IMPORT SS FUNCTIONAL DATA.

# selected func files
datafiles = sorted(glob.glob(os.path.join(
    derivatives_path, f'*task-ss*_space-T1w_desc-preproc_bold.nii.gz')))

# load the data and convert to NumPy array
data = [nib.load(f).get_fdata().astype(np.float32) for f in datafiles]

# Get the number of volumes from the NumPy array.
T = data[0].shape[-1]

# Get the number of volumes in xyz from the NumPy array.
xyz= data[0].shape[:3]
# print(f'data shape: {data[1].shape}')  # expected (x, y, z, T)
#print("len(data)  =", len(data)  if isinstance(data,  (list,tuple)) else "not a list")

print(f"\nSS fMRI data files were identified for {len(data)} runs:")
print("\n".join(f"{i}. {os.path.basename(f)}" for i, f in enumerate(datafiles)))
print(f'\nThe shape of the fMRI data set for each run is: {data[0].shape}')  # expected (x, y, z, T)

In [ ]:
# VISUALIZE FUNCTIONAL DATA FROM EACH SS RUN.

slice_idx = 40

plt.figure(figsize=(20, 10))
for i in range(len(data)):
    plt.subplot(2, 2, i+1)
    plt.imshow(data[i][:,:,slice_idx,0], cmap='gray')
    plt.title(f'Example slice from run {i+1}', fontsize=14)
    plt.axis('off')
plt.show()

In [ ]:
## TIME SERIES INTERPOLATION.

# Resample 4D fMRI time series (with shape `(X, Y, Z, T)`) from the original TR (`orig_tr`)
# to a new TR (`tr`) using internally defined function (see the beginning of the tutorial!).

orig_tr = 1.5
data = [t_series_interpolation(d, orig_tr, tr) for d in data]

# Print some relevant metadata
print(f'Dimensions of the new run 1 time series after interpolation: {data[0].shape}')
print(f'Dimensions of the new run 2 time series after interpolation: {data[1].shape}')
print(f'\nData has {len(data)} runs\n')
print(f'Shape of data from each run is: {data[0].shape}')
print(f'XYZ dimensionality is: {data[0].shape[:3]} (one slice only)')
print(f'N = {data[0].shape[3]} TRs per run')

###(3) Run GLMsingle.

For **every voxel**, GLMsingle:

1.   Selects the best HRF function.
2.   Applies GLMdenoise to remove noise.
3.   Fractional ridge regression.

Together, these three features to maximize data quality at the **individual voxel** and **individual trial** levels.

In [ ]:
# Set options for GLMsingle.
opt = dict()
opt['wantlibrary'] = 1 # yes, we want to use GLMsingles built-in HRF library
opt['wantglmdenoise'] = 1 # yes, we want to apply GLMdenoise
opt['wantfracridge'] = 1 # yes, we want to use Fractional Ridge Regression for single-trial beta estimates
opt['wantfileoutputs'] = [1,1,1,1]
opt['wantmemoryoutputs'] = [1,1,1,1]
glmsingle_obj = GLM_single(opt)

# Print out options for GLMsingle.
print("GLMsingle hyperparameters:")
pprint(glmsingle_obj.params)

In [ ]:
# Do it! (But actually don't becaust it will take too long! Just load in some
# pre-baked outputs for now.

# Because we already have the data processed, the code below will import it.
# However, the code below will also run GLMsingle, if the outputs don't already
# exist.

# Set output directories.
outputdir_glmsingle = os.path.join(derivatives_path, "func-ss-glmsingle")
os.makedirs(outputdir_glmsingle, exist_ok=True)
print("GLMsingle outputs available at:", outputdir_glmsingle)

# Define expected GLMsingle output files
typeA = join(outputdir_glmsingle,'TYPEA_ONOFF.npy')
typeB = join(outputdir_glmsingle,'TYPEB_FITHRF.npy')
typeC = join(outputdir_glmsingle,'TYPEC_FITHRF_GLMDENOISE.npy')
typeD = join(outputdir_glmsingle,'TYPED_FITHRF_GLMDENOISE_RR.npy')

if not exists(typeD):

  # Run GLMsingle.
  results_glmsingle = glmsingle_obj.fit(
      design,           # List of design matrices (one per run)
      data,             # List of 4D BOLD data arrays (X, Y, Z, T)
      stimdur,          # Stimulus duration in seconds
      tr,               # Repetition time (TR) in seconds = 1.0
      outputdir=outputdir_glmsingle  # Where to save the model outputs (A–D)
      )

else:

  # Load GLMsingle outputs.
  typeA = np.load(typeA, allow_pickle=True).item()
  typeB = np.load(typeB, allow_pickle=True).item()
  typeC = np.load(typeC, allow_pickle=True).item()
  typeD = np.load(typeD, allow_pickle=True).item()
  results_glmsingle = {'typed': {'betasmd': typeD}}

###(4) Visualize data quality outputs.

GLMsingle implements some important data quality steps that allow more precise estimates of individual brain responses than standard GLM approaches.

In [ ]:
# Print the shape of the first proc functional run

print(f"data[0] shape: {data[0].shape}")  # (X, Y, Z, T)

# Extract spatial dimensions (X, Y, Z)
xyz = data[0].shape[:3]

# Compute the mean volume across time (TRs) from the first run
meanvol = np.mean(data[0], axis=3)
if len(data) > 1:
    meanvol = 0.5 * (meanvol + np.mean(data[1], axis=3))  # Optional: blend first two runs

# Generate a brain mask by thresholding low-intensity background
thr = np.percentile(meanvol[meanvol > 0], 40) if np.any(meanvol > 0) else np.percentile(meanvol, 40)
brainmask = meanvol > thr

In [ ]:
# Plot quality assurance: betasmd.

# betasmd: is the full set of single-trial beta weights (X x Y x Z x TRIALS). beta weights are arranged in chronological order.

betas = typeD['betasmd']

# For beta maps, average over all trials (axis 3)
if betas.ndim == 4:
    # (X, Y, Z, TRIAL)
    plot_data = np.nanmean(betas, axis=3).astype(float)
elif betas.ndim == 3:
    plot_data = betas.astype(float)
else:
    raise ValueError(f"Unexpected betas shape: {betas.shape}")

slice_idx = 20
sl = plot_data[:, 5:-5, slice_idx].copy()
sl[~brainmask[:, 5:-5, slice_idx]] = np.nan

plt.figure(figsize=(6, 4))
plt.imshow(sl, cmap='RdBu_r', vmin=-5, vmax=5, origin='lower')
plt.colorbar()
plt.title('average GLM betas (runs 1-2)')
plt.axis('off')
plt.show()

In [ ]:
# Plot quality assurance: R2.

# R2: model accuracy expressed in terms of R^2 (percentage).

# For other GLMsingle outputs, reshape to volume
R2 = typeD['R2']

if R2.ndim == 4:
    plot_data = np.nanmean(R2, axis=3).astype(float)
elif R2.ndim == 3:
    plot_data = R2.astype(float)
else:
    raise ValueError(f"Unexpected R2 shape: {R2.shape}")
 # Cut slice (remove 5 voxels on top and bottom in Y to avoid edge effects)

slice_idx = 20;
sl = plot_data[:, 5:-5, slice_idx].copy()
mask_sl = brainmask[:, 5:-5, slice_idx]
sl[~mask_sl] = np.nan

# Plot
plt.figure(figsize=(6, 4))
plt.imshow(sl, cmap='hot', vmin=0, vmax=85, origin='lower')
plt.colorbar()
plt.title('R² (mean across runs)')
plt.axis('off')
plt.show()

In [ ]:
# Plot quality assurance: HRFindex.

# HRFindex: is the 1-index of the best fit HRF. HRFs can be recovered with getcanonicalHRFlibrary(stimdur,tr).

# For other GLMsingle outputs, reshape to volume
R2 = typeD['HRFindex']

if R2.ndim == 4:
    plot_data = np.nanmean(R2, axis=3).astype(float)
elif R2.ndim == 3:
    plot_data = R2.astype(float)
else:
    raise ValueError(f"Unexpected R2 shape: {R2.shape}")
# Cut slice (remove 5 voxels on top and bottom in Y to  avoid edge effects)
slice_idx = 20 # Change slc to visualize different slices.
sl = plot_data[:, 5:-5, slice_idx].copy()
sl[~brainmask[:, 5:-5, slice_idx]] = np.nan

# Plot
plt.figure(figsize=(6, 4))
plt.imshow(sl, cmap='jet', vmin=0, vmax=20, origin='lower')
plt.colorbar()
plt.title('HRF index')
plt.axis('off')
plt.show()

In [ ]:
# Plot quality assurance: FRACvalue.

# FRACvalue: is the fractional ridge regression regularization level chosen for each voxel. Values closer to 1 mean less regularization.

FRAC = typeD['FRACvalue']

# Start plotting
plt.figure(figsize=(12, 8))

if FRAC.ndim == 4:
    plot_data = np.nanmean(FRAC, axis=3).astype(float)
elif FRAC.ndim == 3:
    plot_data = FRAC.astype(float)
else:
    raise ValueError(f"Unexpected FRACvalue shape: {FRAC.shape}")

# Cut slice (remove 5 voxels on top and bottom in Y to avoid edge effects)
slice_idx = 20 # Change slc to visualize different slices.
sl = plot_data[:, 5:-5, slice_idx].copy()
sl[~brainmask[:, 5:-5, slice_idx]] = np.nan

# Plot
plt.figure(figsize=(6, 4))
plt.imshow(sl, cmap='copper', vmin=0, vmax=1, origin='lower')
plt.colorbar()
plt.title('Voxel fraction (mean across runs)')
plt.axis('off')
plt.show()

# **Step 3**: Identify FFA and VWFA regions of interest (ROIs).

**Goal**: Visualize the ROIs that we will be analyzing.  
**We will**: (1) Import previously-defined ROI masks → (2) Import the functional data → (3) Visualize the ROIs on our functional data!

Participants also completed a standard localizer procedure ([fLoc](https://github.com/Learning-and-Neurodevelopment-Lab/fLoc)) that has been **designed to identify person-specific category selective responses in ventral-temporal cortex**. We will use fLoc data to identify ROIs consistent with VWFA and FFA.

In [ ]:
YouTubeVideo("_IQ0QDOukoc", embed=True, width=640, height=320)

Applying it to the final visit will increase the likelihood that we will find the VWFA ROI in this individual child because we are more likely to find word-selectivity after first grade (at the final visit) than before first grade (at the first visit).

In [ ]:
### (1) IMPORT PREVIOUSLY-DEFINED ROI MASKS.

# We used the fLoc experiment to identify an area in the right ventral temporal
# cortex (rvtc) that responded selectively to faces
# (faces>(places+letters+digits)) and another area in left ventral temporal
# cortex (lvtc) that responded selectively letters
# (letters>(faces+places+digits)), t>1.96.

# Define the path to the **roi masks**.
roi_dir = os.path.join(rootdir, "function", f"{subid}12", "func", "func-floc-glmsingle", "contrasts-floc", "ROIs")

# Find all roi masks for face responsive areas, right hemisphere.
face_roi_path   = [f for f in os.listdir(roi_dir) if "faces" in f and f.endswith(".nii.gz")][0]

# Find all roi masks for letter responsive areas, left hemisphere.
letter_roi_path = [f for f in os.listdir(roi_dir) if "letters" in f and f.endswith(".nii.gz")][0]

# Load ROI masks.
faces_roi   = nib.load(os.path.join(roi_dir, face_roi_path)).get_fdata()
letters_roi = nib.load(os.path.join(roi_dir, letter_roi_path)).get_fdata()

# Print filepaths.
print("\nDirectory where the ROI masks are located:\n", roi_dir)
print("\nList of ROI masks found:")
for f in os.listdir(roi_dir):
    print(f)

# Print summary of how many voxels within that ROI.
print(f'\nThere are {np.sum(faces_roi)} voxels in the included FFA ROI')
print(f'There are {np.sum(letters_roi)} voxels in the included VWFA ROI')

In [ ]:
### (2) IMPORT THE FUNCTIONAL SS DATA FOR RUN 1 OF VISIT 12.

# We're only going to import one run because we're just using the functional
# data for visualization.

# Define directory where the preprocessed functional data can be found.
func_dir = os.path.join(rootdir, "function", f"{subid}12", "func")

# Define file path to the preprocessed run 1 functional SS data.
func_template_path = [f for f in os.listdir(func_dir) if "task-ss_run-01_space-T1w_desc-preproc" in f and f.endswith("_bold.nii.gz")][0]

# Load run 1 preprocessed functional data.
fmri_data = nib.load(os.path.join(func_dir, func_template_path)).get_fdata()[:,:,:,0]

# Print filepaths.
print("\nDirectory where visit 12 functional data are located:\n", func_dir)
print("\nFilename for functional SS data for run 1 of visit 12:\n", func_template_path)

In [ ]:
### (3) VISUALIZE FLOC ROI MASKS ON SS FUNCTIONAL DATA.

# Set slice_idx to visualize slices of interest. TIP: Change 20 to other
# numbers, e.g., 21, and then rerun this block to surf through the axial slices!
slice_idx = 21

# Set the figure size.
plt.figure(figsize=(5, 4))

# Select the fmri data for the slice of interest.
bg = fmri_data[:,:,slice_idx].T

# Plot the fmri data of the selected slice using a gray colormap.
plt.imshow(bg, cmap='gray');

# Select the ROI face mask data for the slice of interest.
face_roi = faces_roi[:,:,slice_idx].T

# Plot the ROI face mask in "autumn" to display in red.
plt.imshow(np.where(face_roi>0, 1, np.nan), cmap='autumn', alpha=0.95);

# Select the ROI letter mask data for the slice of interest.
letter_roi = letters_roi[:,:,slice_idx].T

# Plot the ROI letter mask in "cool" to display in cyan.
plt.imshow(np.where(letter_roi>0, 1, np.nan), cmap='cool', alpha=0.95);

# Clean up the disply by removing the axes labels.
plt.axis('off');

# **Step 4**: Extract brain response estimates for face and letter images from voxels in the FFA and VWFA.

**Goal**: Calculate mean and variance for brain responses to face and letter images in the VWFA and FFA ROIs.

**We will**: (1) Load SS beta-weights for all runs and visits. → (2) Reload the fLoc ROI masks. → (3) Compute statistics for plotting.

In [ ]:
## (1) Load beta-weights for all runs and visits.

designs = []
betas = []

for v in visits:
    print(f"🔄 Loading visit {v}...")
    vp = visit_dir(v)
    design_path = vp / 'DESIGNINFO.mat'
    beta_path = vp / 'TYPED_FITHRF_GLMDENOISE_RR.mat'

    if not design_path.exists() or not beta_path.exists():
        print(f"❌ Missing files in visit {v}")
        continue

    try:
        designs.append(load_designinfo(design_path))
        betas.append(load_modelmd(beta_path))
    except Exception as e:
        print(f"❌ Error loading visit {v}: {e}")
        continue

# --- Final report ---
if betas:
    print(f"\n✅ Successfully loaded {len(betas)} visits")
    print(f"Beta shape: {betas[0].shape}")
    print(f"Stimorder length: {len(designs[0]['stimorder'])}")
else:
    print("⚠️ No data loaded.")

In [ ]:
## (2) Reload the fLoc ROI masks.

target_xyz = (58, 73, 57)  # e.g., known from previous data (58, 73, 57)

# ---- Reload ROI masks ----
roifiles = sorted([f for f in os.listdir(roi_dir) if f.endswith(".nii.gz") and "nvox=0" not in f])
roi_masks, roi_names = [], []

for f in roifiles:
    img = nib.load(os.path.join(roi_dir, f))
    # Optional: convert to canonical RAS+ orientation for consistency
    img = nib.as_closest_canonical(img)
    arr = np.asanyarray(img.dataobj)   # Expected shape: (58, 73, 57)
    mask = arr > 0
    assert mask.ndim == 3
    roi_masks.append(mask)
    roi_names.append(f.replace(".nii.gz",""))

print(f" Reloaded {len(roi_masks)} ROIs, all shape = {target_xyz}")

In [ ]:
## (3) Compute statistics and export for plotting later.

# ---- Condition grouping (2 categories Faces / Letters) ----
# To select only Faces / Letters, replace above with:
coilist    = [1, 9]
coi_labels = ['Faces','Letters']

n_roi   = len(roi_masks)
n_cond  = len(coilist)
n_visit = len(betas)

m  = np.zeros((n_visit, n_cond, n_roi))
md = np.zeros((n_visit, n_cond, n_roi))
sd = np.zeros((n_visit, n_cond, n_roi))
se = np.zeros((n_visit, n_cond, n_roi))
d  = {}

# ---- Extract voxel statistics within each ROI per condition per visit ----
for r, roihere in enumerate(roi_masks):
    for c, cond_id in enumerate(coilist):
        for v in range(n_visit):
            modelmd   = betas[v]
            stimorder = designs[v]['stimorder'].astype(int).ravel()
            idx = np.where(stimorder == cond_id)[0]
            if idx.size == 0:
                raise ValueError(f"No trials for cond_id {cond_id} at visit {v}")

            if modelmd.shape[-1] == len(stimorder):          # shape = (X,Y,Z,n_trials)
                beta = modelmd[:, :, :, idx].mean(axis=3)
            elif modelmd.shape[0] == len(stimorder):         # shape = (n_trials,X,Y,Z)
                beta = modelmd[idx, ...].mean(axis=0)
            else:
                raise ValueError(f"Unexpected beta shape: {modelmd.shape}")

            beta, mask = align_xyz(beta, roihere.astype(bool),
                                   tag=f"visit={v}, roi={roi_names[r]}, cond={coi_labels[c]}")
            beta_flat = beta[mask]

            m[v, c, r]  = beta_flat.mean()
            md[v, c, r] = np.median(beta_flat)
            sd[v, c, r] = beta_flat.std(ddof=0)
            se[v, c, r] = sd[v, c, r] / np.sqrt(beta_flat.size)
            d[(v, c, r)] = beta_flat

print("✅ All statistics extracted without axis mismatch.")


# **Step 5**: Visualize trajectories of brain responses to faces and letters throughout first grade.

**Goal**: Plot the mean betas for face and letter images within our ROIs at each of the 12 visits.  


**We will**: (1) Create a data dictionary to store the statistics and (2) Plot them!

In [ ]:
# Create a data dictionary for the statistics that will be plotted.

all_roi_data = {}  # Initialize a dictionary to hold all data

for r, roi_name in enumerate(roi_names):

    # Create a dictionary for the current ROI's data
    roi_data = {}

    # Convert numpy array slices to pandas DataFrames and store them
    roi_data['m'] = pd.DataFrame(m[:, :, r].tolist(), columns=coi_labels)
    roi_data['sd'] = pd.DataFrame(sd[:, :, r].tolist(), columns=coi_labels)
    roi_data['se'] = pd.DataFrame(se[:, :, r].tolist(), columns=coi_labels)
    roi_data['md'] = pd.DataFrame(md[:, :, r].tolist(), columns=coi_labels)

    # Add the current ROI's data dictionary to the main dictionary
    all_roi_data[roi_name] = roi_data

print("✅ DataFrames created and stored in memory.")

In [ ]:
# (2) Plot!

## -->> Simulate scenario with longitudinal sampling that's more or less dense.

n_samples = 12 # You can change this number between 1 and 12 (e.g., 2, 6, 12)

idx_visits = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

# Ensure n_samples is not greater than the number of available visits
if n_samples > len(idx_visits):
    n_samples = len(idx_visits)

months = np.sort(np.array(random.sample(idx_visits, n_samples)))
print(f"Randomly selected {n_samples} months for plotting: {months}")

# Process and plot for each ROI using the data stored in the dictionary
for roi_name, data in reversed(all_roi_data.items()):

    print(f"Processing {roi_name}...")

    try:
        # Retrieve dataframes from the dictionary
        df_m = data['m']
        df_md = data['md']
        df_sd = data['sd']
        df_se = data['se']

        # Create plot using the randomly selected months
        # Note: .iloc uses 0-based indexing, so we subtract 1 from the month numbers
        plot_roi_results(df_m.iloc[months-1],
                         df_md.iloc[months-1],
                         df_sd.iloc[months-1],
                         df_se.iloc[months-1],
                         roi_name);

    except Exception as e:
       print(f" ")

print("\nAll ROIs processed!")


# **THE END**: Congratulations on your first analysis with dense longitudinal neuroimaging data! And, thank you!